In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

em_data_filepath='emdat_public_2022_08_11_query_uid-IKGfZc.xlsx'

dfb = pd.read_excel(em_data_filepath, engine='openpyxl',skiprows=6,sheet_name = 'emdat data')

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## East Africa Drought

In [2]:
dfb_dr=dfb[dfb['Disaster Type']=='Drought']

ea_cntr=['Djibouti', 'Eritrea','Ethiopia','Kenya','Rwanda', 'Somalia','South Sudan','Sudan (the)','Tanzania, United Republic of']

#dfb1_ea=dfb1[dfb1['Country'].str.contains(ea_cntr)]
dfb_dr_ea=dfb_dr[dfb_dr['Country'].isin(ea_cntr)]
dfb_dr_ea.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 77 to 4601
Data columns (total 50 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Dis No                                     62 non-null     object 
 1   Year                                       62 non-null     int64  
 2   Seq                                        62 non-null     int64  
 3   Glide                                      23 non-null     object 
 4   Disaster Group                             62 non-null     object 
 5   Disaster Subgroup                          62 non-null     object 
 6   Disaster Type                              62 non-null     object 
 7   Disaster Subtype                           62 non-null     object 
 8   Disaster Subsubtype                        0 non-null      object 
 9   Event Name                                 0 non-null      object 
 10  Country                  

In [8]:
em_data_location_filepath='em_dat_ea_drought_adm2.xlsx'

dfb = pd.read_excel(em_data_location_filepath, engine='openpyxl')

dfb['count']=dfb['geob_adm2_list'].str.len()
dfb

,Dis No,geob_adm2_list,count
0,1999-9388-ERI,"['ERI-ADM2-3_0_0-B20', 'ERI-ADM2-3_0_0-B21', '...",543
1,1999-9388-ETH,"['ETH-ADM2-3_0_0-B5', 'ETH-ADM2-3_0_0-B33', 'E...",1092
2,1999-9388-DJI,"['DJI-ADM2-3_0_0-B10', 'DJI-ADM2-3_0_0-B7', 'D...",233
3,1999-9388-SDN,"['SDN-ADM2-3_0_0-B78', 'SDN-ADM2-3_0_0-B79', '...",713
4,1999-9388-RWA,"['RWA-ADM2-3_0_0-B1', 'RWA-ADM2-3_0_0-B2', 'RW...",63
...,...,...,...
57,1996-9326-TZA,"['TZA-ADM2-3_0_0-B165', 'TZA-ADM2-3_0_0-B163',...",2351
58,1996-9089-RWA,['RWA-ADM2-3_0_0-B8'],21
59,1996-9220-SDN,"['SDN-ADM2-3_0_0-B52', 'SDN-ADM2-3_0_0-B46', '...",222
60,1998-9237-ETH,"['ETH-ADM2-3_0_0-B4', 'ETH-ADM2-3_0_0-B27', 'E...",1619


In [ ]:
dfb_dr_ea_time=dfb_dr_ea[['Dis No','Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month','End Day']]

dfb_dr_ea_time.info()
#pd.set_option('display.max_columns', 8)
#pd.set_option('display.max_rows', 200)

In [ ]:


dfb_dr_ea_time=dfb_dr_ea[['Dis No','Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month','End Day']]
#dfb_dr_ea_time['Start Month'].tolist()
dfb_dr_ea_time['Start Month']=dfb_dr_ea_time['Start Month'].fillna(1.0)
dfb_dr_ea_time['Start Day']=dfb_dr_ea_time['Start Day'].fillna(1.0)

dfb_dr_ea_time['End Month']=dfb_dr_ea_time['End Month'].fillna(12.0)

def last_day_month(row):
    firstday_month = datetime(int(row['End Year']), int(row['End Month']), 1)
    lastday_month = firstday_month + pd.DateOffset(months=1) - pd.DateOffset(days=1)
    lastday_month_str =lastday_month.strftime("%d")
    return lastday_month_str

 
for idx,row in dfb_dr_ea_time.iterrows():
    dfb_dr_ea_time.loc[idx,'End Day'] = last_day_month(row)    
    #dfb_dr_ea_time.loc[idx,'end_day'] = last_day_month(row)    

    
#dfb_dr_ea_time['End Day']=dfb_dr_ea_time['Start Day'].fillna(28.0)


dfb_dr_ea_time['Start Year']=dfb_dr_ea_time["Start Year"].astype(str)
#dfb_dr_ea_time['Start Month']=dfb_dr_ea_time["Start Month"].astype(str)
dfb_dr_ea_time['Start Month']=dfb_dr_ea_time["Start Month"].apply(lambda x: str(int(x)).zfill(2))
dfb_dr_ea_time['End Year']=dfb_dr_ea_time["End Year"].astype(str)
#dfb_dr_ea_time['End Month']=dfb_dr_ea_time["End Month"].astype(str)
dfb_dr_ea_time['End Month']=dfb_dr_ea_time["End Month"].apply(lambda x: str(int(x)).zfill(2))

In [ ]:
#pd.date_range('2014-10-10','2016-01-07', freq='MS').strftime("%Y-%m").tolist()

def create_daterange_list(row):
    start_date_str=row['Start Year']+'-'+row['Start Month']+'-'+str(int(row['Start Day']))
    end_date_str=row['End Year']+'-'+row['End Month']+'-'+row['End Day']
    date_str_list=pd.date_range(start_date_str,end_date_str, freq='D').strftime("%Y-%m-%d").tolist()
    return date_str_list


date_str_list_cont=[]
for idx,row in dfb_dr_ea_time.iterrows():
    #dfb_dr_ea_time.at[idx,'date_str_list'] = create_daterange_list(row)
    #print(idx)
    #print(row)
    date_str_list=create_daterange_list(row)
    date_str_list_cont.append(date_str_list)
    
#dfb_dr_ea_time

#df_dr=dfb_dr_ea_time.explode('date_str_list')

#df_dr1=df_dr.drop_duplicates('date_str_list')
dfb_dr_ea_time['date_str_list']=date_str_list_cont
df_dr=dfb_dr_ea_time.explode('date_str_list')
df_dr1=df_dr.drop_duplicates('date_str_list')

In [ ]:
df_dr1['dr']=1
#df_dr1=df_dr.drop_duplicates('date_str_list')
df_dr1['date']=df_dr1['date_str_list'].apply(pd.to_datetime)
df_dr1['doy']=df_dr1['date'].dt.dayofyear
df_dr1['year']=df_dr1['date'].dt.year
df_dr2=df_dr1[['year','doy','dr']]
df=pd.pivot_table(df_dr2, values='dr', index=['year'],columns=['doy'])
df1=df.rename_axis(None)

df2=df1.fillna(2)

cols=df2.columns
df2[cols] = df2[cols].applymap(np.int64)
df3=df2.iloc[:, 0:60]
df4=df3.reset_index()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import six
from datetime import datetime


### functions for whole column, row editing
def set_align_for_column(table, col, align="left"):
    cells = [key for key in table._cells if key[1] == col]
    for cell in cells:
        table._cells[cell]._loc = align


def set_width_for_column(table, col, width):
    cells = [key for key in table._cells if key[1] == col]
    for cell in cells:
        table._cells[cell]._width = width
        
def set_height_for_row(table, row, height):
    cells = [key for key in table._cells if key[0] == row]
    for cell in cells:
        table._cells[cell]._height = height

def colorcell(tablerows,tablecols,cellDict):
    allcells=[(x,y) for x in tablerows[1:] for y in tablecols[1:]]
    for alcls in allcells:
        if int(cellDict[alcls]._text.get_text()) ==0:
            cellDict[alcls].set_facecolor('#009600')
        elif int(cellDict[alcls]._text.get_text()) == 1:
            cellDict[alcls].set_facecolor('#64C800')
        else:
            cellDict[alcls].set_facecolor('#961414')

def removeaqivalue(tablerows,tablecols,mpl_table):
    allcells=[(x,y) for x in tablerows[1:] for y in tablecols[1:]]
    for alcls in allcells:
        mpl_table._cells[alcls]._text.set_text('')

def set_height_for_row_except_head(table, rowlist, height):
    cells_list=[]
    for row in rowlist:
        cells = [key for key in table._cells if key[0] == row]
        cells_list.append(cells)
    for cells in cells_list:
        for cell in cells:
             table._cells[cell]._height = height

### funciton for table creation
def render_mpl_table(data, col_width=1.0, row_height=1.625, font_size=12,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=['' for _ in range(366)],cellLoc='center', **kwargs)
    set_align_for_column(mpl_table, col=0, align="left")
    cellDict=mpl_table.get_celld()
    noofcol=8
    noofrow=len(cellDict)/noofcol
    set_width_for_column(mpl_table, 0, 0.1)
    #set_width_for_column(mpl_table, 1, 0.02)
    #set_width_for_column(mpl_table, 2, 0.02)
    #set_width_for_column(mpl_table, 3, 0.02)
    #set_width_for_column(mpl_table, 4, 0.02)
    set_height_for_row(mpl_table, 0, 0.01)
    set_height_for_row_except_head(mpl_table, np.arange(1,len(data.index)), 0.012)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)
    cellDict=mpl_table.get_celld()
    tablerows=np.arange(0,len(data.index)+1)
    tablecols=np.arange(0,len(data.columns))
    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_linewidth(0)
        #cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
            colorcell(tablerows,tablecols,cellDict)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
#     headings=data.columns
#     plt.text(0.07, 0.95,'Years', horizontalalignment='left',fontsize=14,fontweight='bold',
#              color='w', verticalalignment='center', transform =ax.transAxes)
#     plt.text(0.6, 0.965,'Jan', horizontalalignment='left',fontsize=10,fontweight='bold',
#              color='w', verticalalignment='center',rotation=90, transform =ax.transAxes)
#     plt.text(0.715, 0.965,'Feb', horizontalalignment='left',fontsize=10,fontweight='bold',
#              color='w', verticalalignment='center',rotation=90, transform =ax.transAxes)
#     plt.text(0.815, 0.965,'Mar', horizontalalignment='left',fontsize=10,fontweight='bold',
#              color='w', verticalalignment='center',rotation=90, transform =ax.transAxes)
#     plt.text(0.925, 0.965,'Apr', horizontalalignment='left',fontsize=10,fontweight='bold',
#              color='w', verticalalignment='center',rotation=90, transform =ax.transAxes)
    removeaqivalue(tablerows,tablecols,mpl_table)
    return ax


In [ ]:
%matplotlib notebook

### Define the picture size and remove the ticks
fig = plt.figure()
fig.set_size_inches(26,18)
table=fig.add_axes([0.08, 0.02, 0.55, 0.9], frame_on=False) 
table.xaxis.set_ticks_position('none')
table.yaxis.set_ticks_position('none') 
table.set_xticklabels('')
table.set_yticklabels('')

render_mpl_table(df4, header_columns=1, col_width=0.002,ax=table)
plt.show()

In [ ]:
df4